In [1]:
!pip install flask

In [2]:
!pip install flask pandas

In [ ]:
###upload dataset and return dataset id###

from flask import Flask, request, jsonify
import os
import pandas as pd
import datetime

app = Flask(__name__)

datasets = {}  # Dictionary to store datasets with unique IDs
log_file = "dataset_log.xlsx"
log_columns = ["dataset_id", "dataset_name", "description", "file_path"]

if not os.path.exists(log_file):
    df = pd.DataFrame(columns=log_columns)
    df.to_excel(log_file, index=False)

def generate_dataset_id():
    return f"ID_{len(datasets) + 1:03d}"  # Generates ID_001, ID_002, etc.

@app.route("/create_dataset", methods=["POST"])
def create_dataset():
    try:
        if 'textfile' not in request.files:
            return jsonify({"error": "Missing textfile parameter"}), 400

        textfile = request.files['textfile']
        content = textfile.read().decode("utf-8").strip().split("\n")
        
        data_dict = {}
        for line in content:
            key, value = line.split(":", 1)
            data_dict[key.strip()] = value.strip()
        
        dataset_name = data_dict.get("dataset_name")
        description = data_dict.get("description")
        file_path = data_dict.get("textfilepath")
        project_type = data_dict.get("project_type")
        
        if not dataset_name or not description or not file_path:
            return jsonify({"error": "Missing required parameters in text file"}), 400
        
        dataset_id = generate_dataset_id()
        datasets[dataset_id] = {
            "datasetName": dataset_name,
            "description": description,
            "filePath": file_path,
            "project_type":project_type
        }

        # Save file in GENAI_HOME folder
        os.makedirs("GENAI_HOME", exist_ok=True)
        timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
        file_name = f"{timestamp}_{os.path.basename(textfile.filename)}_{dataset_id}.txt"
        file_save_path = os.path.join("GENAI_HOME", file_name)
        with open(file_save_path, "w") as f:
            f.write("\n".join(content))

        # Log dataset info to Excel
        df = pd.read_excel(log_file)
        new_row = pd.DataFrame([{"dataset_id": dataset_id, "dataset_name": dataset_name, "description": description, "file_path": file_save_path,"project_type":project_type}])
        df = pd.concat([df, new_row], ignore_index=True)
        df.to_excel(log_file, index=False)

        return jsonify({"datasetId": dataset_id, "message": "Dataset created successfully"}), 201
    
    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Feb/2025 19:57:52] "POST /create_dataset HTTP/1.1" 201 -


In [ ]:
from flask import Flask, request, jsonify
import os
import pandas as pd
import datetime
import random  # ✅ Import the missing module

app = Flask(__name__)

GENAI_HOME = "GENAI_HOME"
os.makedirs(GENAI_HOME, exist_ok=True)

EXCEL_FILE = "metrics_results.xlsx"

def read_config_file(file_path):
    """Reads the text file and extracts parameters."""
    config = {}
    with open(file_path, "r") as f:
        for line in f:
            key, value = line.strip().split(": ", 1)  # Split on ": "
            config[key] = value.strip()
    return config

@app.route("/evaluate_metrics", methods=["POST"])
def evaluate_metrics():
    try:
        # Check if file is uploaded
        if 'configfile' not in request.files:
            return jsonify({"error": "Missing 'configfile' parameter"}), 400

        file = request.files['configfile']

        # Save temporarily & Read file
        temp_file_path = os.path.join(GENAI_HOME, file.filename)
        file.save(temp_file_path)
        config = read_config_file(temp_file_path)

        dataset_id = config.get("dataset_id")
        project_type = config.get("project_type")
        metrics = config.get("metrics", "").split(", ")

        if not dataset_id or not project_type or not metrics:
            return jsonify({"error": "Missing required parameters in config file"}), 400

        # Find the corresponding dataset file
        dataset_files = [f for f in os.listdir(GENAI_HOME) if f.endswith(f"_{dataset_id}.txt")]
        if not dataset_files:
            return jsonify({"error": f"No dataset found for dataset_id: {dataset_id}"}), 404

        dataset_file = os.path.join(GENAI_HOME, dataset_files[0])

        # Simulate metrics evaluation
        results = []
        for metric in metrics:
            score = round(random.uniform(0, 1), 2)  # ✅ Fix: Import random module
            reasoning = f"Placeholder reasoning for {metric}"
            results.append({"dataset_id": dataset_id, "metric": metric, "score": score, "reasoning": reasoning})

        # Store results in Excel
        df = pd.DataFrame(results)
        if os.path.exists(EXCEL_FILE):
            existing_df = pd.read_excel(EXCEL_FILE)
            df = pd.concat([existing_df, df], ignore_index=True)

        df.to_excel(EXCEL_FILE, index=False)

        return jsonify({"status": "success", "message": "Metrics evaluated successfully", "results": results})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

if __name__ == "__main__":
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Feb/2025 17:48:52] "POST /evaluate_metrics HTTP/1.1" 200 -
